<img src="../assets/logos/Logo_shoot.png" width=500></img>
# Aprendizaje profundo
Tu primer modelo hizo disparos aleatorios y tu segundo modelo trató de igualar el juego actual con juegos anteriores; sin embargo, si ningún juego anterior coincidió, entonces fueron solo intentos al azar. Este modelo tendrá un mejor desempeño al __aprender__ el patrón de distribución de los barcos, no solo memorizándolos.

## Datos
Usarás datos de juegos anteriores para entrenar el modelo. Primero debemos descargar los datos del juego maestro archivado ejecutando: (ten en cuenta que los datos se están actualizando constantemente a medida que los juegos se juegan)
```bash
./bin/download_data.sh # Download, parse data, create sample data
```
a continuación copia el archivo de datos de muestra en el directorio de simulación del código para este modelo
```bash
cp ./data/data.min.json ./containers/Shoot/cnn/mock/data/data.json
```
Ahora podrás probar tu modelo entrenando localmente

A continuación debemos cargar los datos a nuestro bucket s3 para que podamos entrenar el modelo de producción en él. Ejecuta:
```bash
./bin/upload_data.sh
```

## Código
El código está en /containers/Shoot/CNN y contiene los siguientes archivos
    
- ### ./containers/Shoot/CNN/Makefile
Makefile para crear el archivo tar.gz del código para el entorno de trabajo de sagemaker

- ### [./containers/Shoot/CNN/host.py](/edit/GameDayRepo/containers/Shoot/CNN/host.py)
El código para generar inferencias (no ejecutes este archivo directamente). Además de ejecutar nuestro modelo, también colocará una etiqueta en el atributo de la sesión que usaremos en athena para comparar diferentes modelos. Puedes cambiar el valor de la etiqueta (lo cual deberías hacer por cada modelo) editando el archivo host.py.

- ### [./containers/Shoot/CNN/test.py](/edit/GameDayRepo/containers/Shoot/CNN/test.py)
código para probar los archivos host.py y train.py localmente. Crea una simulación de tiempo de ejecución de sagemaker para ejecutar train.py y luego test.py. Puedes editar este archivo para personalizar tu prueba. Si recibes un mensaje acerca de dependencias faltantes (como mxnet) simplemente instálalas con:
```shell
pip install mxnet # for example
```

- ### [./containers/Shoot/CNN/train.py](/edit/GameDayRepo/containers/Shoot/CNN/train.py)
el código de entrenamiento (no ejecutes este archivo directamente)


A continuación, ejecuta el script de prueba localmente para confirmar que los scripts funcionan. Para hacer esto debes descargar el conjunto de datos y crear un conjunto de datos de muestra. Haz esto ejecutando el script de descarga en el directorio bin. 
```shell
./bin/download_data.sh #downloads data from master Archive, parses it, and also create data sample
```

luego en el directorio del código (/containers/Shoot/CNN) ejecuta:
```shell
mkdir -p mock/data 
cp ../../../data/data.min.json mock/data/data.json #copy the data sample to the mock directory
```
Ahora puedes ejecutar el script de prueba:
```shell
python ./test.py
```

## Configurar
Antes de poder implementar tu segundo modelo, debes configurar la implementación. Como se describe en el [Cuadernillo final](../Endpoint_Reference.ipynb), editarás para la configuración de la implementación en [/shoot-config.json](/edit/GameDayRepo/shoot-config.json). deberás editar el siguiente campo según los siguientes valores

|campo de parámetro|Valor|descripción|
|---|---|---|
|trainsourcefile|`shoot-CNN.tar.gz`|código para nuestro modelo|
|hostsourcefile|`shoot-CNN.tar.gz`|un código para nuestro modelo|
|hyperparameters.epochs|2|Máx. épocas para las cuales entrenar|
|hyperparameters.warm_up|1|Cuántas épocas esperar para comenzar a detenerse anticipadamente|
|hyperparameters.patience|1|Cuántas épocas sin mejoras antes de detenerse anticipadamente|
|hyperparameters.learning_rate|0.005||
|hyperparameters.width|32|Cuántos filtros de características en cada capa|
|hyperparameters.depth|4|cuántas capas|
|hyperparameters.batch_size|1024|tamaño del lote para entrenamiento|
|metrics|[{Name:"Validation",Regex:"Testing loss: (.*?);"},{Name:"Training",Regex:"Training loss: (.*?);"},{Name:"Throughput",Regex:"Throughput=(.*?);"}]|Indicadores para enviar a registros de cloudwatch|
|traininstancetype|"ml.p3.2xlarge"|tipo de instancia con la cual entrenar|
|channels|{"train":{"path":"all/data.json"}}|ruta hacia los datos en bucket de datos|

## Implementar
1. sigue las instrucciones desde la referencia del punto final para aplicar tu código, y luego transferir tus cambios al módulo remoto.
1. Descarga, analiza y carga los datos de entrenamiento ejecutando los scripts download_data.sh y upload_data.sh
```shell
./bin/download_data.sh
./bin/upload_data.sh
```
2. indica a operaciones que implementen tus cambios


## Evaluación de aprendizaje
Después de que tu modelo se haya ejecutado, hay dos maneras en las que desearás evaluar tu modelo
    1. Validation Performance: This is how well your model did on the hold out data set.
    2. Training Throughput: how many data items can it train on per second.

para ver estos indicadores:
1. Visita los [trabajos de entrenamiento de sagemaker](https://us-west-2.console.aws.amazon.com/sagemaker/home?#/jobs) en
1. Selecciona el trabajo de entrenamiento que deseas evaluar (probablemente el más reciente)
<img src="../assets/sagemaker-training-jobs.jpg" width=500></img>
2. desplázate hacia abajo hasta que veas la tarjeta del monitor. Verás tres opciones
<img src="../assets/sagemaker-monitor.jpg" width=500></img>
    1. Ver indicadores de Algorith: abrirá cloudwatch y puedes ver un diagrama de los indicadores de tu modelo. Para este modelo, puedes ver la puntuación de entrenamiento, la puntuación de validación y el desempeño.
    2. Ver registros: Te llevará a los registros de cloudwatch para ver la salida de registros del contenedor de modelos (útil para hacer depuración de errores).
    3. Ver indicadores de Algorith: abrirá cloudwatch y puedes ver un diagrama de los indicadores. (uso de CPU, uso de GPU, uso de memoria, etc...)
    
### Validación del desempeño
El modelo que entrenaste utiliza la [puntuación DICE](https://forums.fast.ai/t/understanding-the-dice-coefficient/5838) que será un número entre 0,0 y 1,0 en el que 1,0 representa un modelo perfecto. Querrás que tu modelo tenga una puntuación de dados lo más alta posible. Aumentar los hiperparámetros de ancho y profundidad hará que tu modelo sea más poderoso y le dará una puntuación de dados más alta (pero llevará más tiempo entrenarlo)

### Resultado del entrenamiento
Tu primer trabajo de entrenamiento debería haber entrenado para solo algunas épocas. Esto te dará una buena estimación del resultado del entrenamiento que puedes usar para ajustar para cuántas épocas entrenar a tu modelo a continuación. Demasiado bajo y tu modelo no aprende lo suficiente, demasiado largo y tu modelo nunca terminó a tiempo para ganar puntos para ti. Como equipo deberás decidir por cuánto tiempo deseas entrenar a un modelo. Observa que aumentar los hiperparámetros de ancho y profundidad disminuirá tu rendimiento, pero usar un tipo de instancia mayor aumentará el rendimiento. 

## Desempeño de la implementación
A continuación, monitorea el desempeño de tu modelo. Consulta [Athena](../Athena.ipynb) para obtener más detalles sobre cómo configurar Athena (reemplaza los valores entre `<>` como en las instrucciones de Athena). La siguiente consulta mostrará el porcentaje ganado por tipo de disparo.

```sql
SELECT type,
         sum(won)/(sum(won)+sum(lost)) as won,
         sum(lost)/(sum(won)+sum(lost)) as lost
FROM 
    (SELECT json_extract(session,'$.shootType') AS type,
        CASE
            WHEN winner = '<Team-Name>' THEN 1.0
            ELSE 0.0
        END AS won,
        CASE
            WHEN winner != '<Team-Name>' THEN 1.0
            ELSE 0.0
        END AS lost
    FROM 
        (SELECT teama.session AS session,
         teama.teamname AS team,
         winner
        FROM "<Game-data-catalog>"."<Game-data-bucket>"
        UNION
        SELECT teamb.session AS session,
         teamb.teamname AS team,
         winner
        FROM "<Game-data-catalog>"."<Game-data-bucket>" )
        WHERE team = '<Team-Name>' )
    GROUP BY  type
```

Intenta editar los hiperparámetros o el código para mejorar los resultados de tu desempeño.